In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import os
import time

In [2]:
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
def generator(noise_dim):
    z = layers.Input(shape=noise_dim)
    x = layers.Dense(units=4*4*1024)(z)
    x = layers.Reshape((4,4,1024))(x)
    for filter_size in [512,256,128,3]:
        x = layers.Conv2DTranspose(filters=filter_size,
               kernel_size=5, strides=2, padding='same')(x)
        x = layers.LeakyReLU(0.2)(x)
        x = layers.BatchNormalization()(x)
    return Model(inputs = z, outputs=x)

In [4]:
def discriminator():
    img = layers.Input(shape=[64,64,3])
    x = layers.Conv2D(filters=128, kernel_size=5, strides=2,
                      padding='same')(img)
    for filter_size in [256, 512,1024]:
        x = layers.Conv2D(filters=filter_size, kernel_size=5,
                          strides=2, padding='same')(x)
        x = layers.LeakyReLU(0.2)(x)
        x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1)(x)
    return Model(inputs = img, outputs = x)

In [5]:
G = generator(100)
G.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense (Dense)               (None, 16384)             1654784   
                                                                 
 reshape (Reshape)           (None, 4, 4, 1024)        0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 512)        13107712  
 nspose)                                                         
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 8, 8, 512)         0         
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 512)        2048      
 ormalization)                                               

In [6]:
D =discriminator()

In [7]:
D.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 128)       9728      
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 batch_normalization_4 (Batc  (None, 16, 16, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 512)         3277312   
                                                           

In [8]:
input_path = "./celebfaces"
def load_real_samples():
    
    # Loading Dataset
    data = np.load(os.path.join(input_path, 'img_celeba.npz'))
    train_x = data['arr_0']
    
    train_x = train_x.astype('float32')
    train_x = (train_x - 127.5) / 127.5
    return train_x

# Loading Dataset
dataset = load_real_samples()


In [9]:
def tf_resize(img):
    return tf.image.resize_with_pad(img, 64,64)

In [10]:
dataset1 = tf.map_fn(fn=tf_resize, elems=dataset)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [11]:
BUFFER_SIZE = 60000
BATCH_SIZE = 64
noise_dim = 100

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(dataset1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [13]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [14]:
intermediate_D = Model(D.inputs, D.get_layer('conv2d_2').output)
def generator_loss(real_output, fake_output):
    features_fake = tf.reduce_mean(intermediate_D(fake_output))
    features_real = tf.reduce_mean(intermediate_D(real_output))
    
    return cross_entropy(tf.ones_like(fake_output), fake_output) \
               + tf.square(tf.norm(features_fake-features_real))

In [15]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [16]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [17]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [18]:
total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")

In [19]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = G(noise, training=True)
        
        real_output = D(images, training=True)
        fake_output = D(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        #gen_loss = generator_loss(real_output, fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        total_loss = gen_loss + disc_loss
    gradients_of_generator = gen_tape.gradient(gen_loss, G.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, D.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, G.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, D.trainable_variables))
    total_loss_tracker.update_state(total_loss)

In [20]:
def save_images(epoch, num_img=3, latent_dim=100):
    random_latent_vectors = tf.random.normal(shape=(num_img, latent_dim))
    generated_images = G(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()
    for i in range(num_img):
        img = tf.keras.preprocessing.image.array_to_img(generated_images[i])
        img.save("generated_img_%03d_%d.png" % (epoch, i))

In [21]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()
        print('starting epoch {}'.format(epoch))
        
        for image_batch in train_dataset:   
            if image_batch.shape[0]==BATCH_SIZE:
                train_step(image_batch)
        print ('Time for epoch {} is {} sec, loss {}'.format(epoch + 1, time.time()-start, total_loss_tracker.result()))
        if epoch % 5 == 0:
            save_images(epoch)

In [ ]:
train(train_dataset, 21)

starting epoch 0
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
